# Climate Profiles 2

### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [1]:
from typing import Tuple
from typing import Any, Dict

import numpy as np
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm  # Progress bar

import climakitae as ck
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv, retrieve_profile_data, _filter_by_ssp, _handle_approach_params
from climakitae.util.warming_levels import get_gwl_at_year
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.core.data_interface import (
    get_data_options,
    get_subsetting_options,
    get_data,
)

import warnings
warnings.filterwarnings("ignore")

# import functions needed for testing
from climakitae.core.constants import UNSET
from climakitae.core.paths import VARIABLE_DESCRIPTIONS_CSV_PATH
from climakitae.explore.typical_meteorological_year import is_HadISD
from climakitae.explore.standard_year_profile import (
    _get_clean_standardyr_filename,
    _check_stations,
    _check_cached_area,
    _check_lat_lon,
    match_str_to_wl,
)
from climakitae.util.utils import read_csv_file
from climakitae.util.warming_levels import get_gwl_at_year

In [2]:
variable = "Air Temperature at 2m"
units = "degF"
qtile = 0.5
warming_levels = [1.5]
no_delta = True

# warming level window size
valid_window = 5
invalid_window_1 = 2
invalid_window_2 = 5.5

# station name options
station_list = [
    "Sacramento Executive Airport (KSAC)",
    "Santa Barbara Municipal Airport (KSBA)",
]
station_name = ["Sacramento Executive Airport (KSAC)"]
custom_name = ["Custom Station Name"]
mixed_list = [
    "Custom Station Name",
    "Santa Barbara Municipal Airport (KSBA)",
]
custom_list = [
    "Custom Name 1",
    "Custom Name 2",
]

# lat lon
# latitude = 34.4041
# longitude = -121.5160 # no data found for these lat and lon bounds!
latitude = 34.19966
longitude = -118.36543  # no data found for these lat and lon bounds!

# cached area
area_name = "Los Angeles County"

### Time-based file modification

Valid inputs to test
- "approach" = "Time" with a "centered_year" between 2015 and 20199
- no "warming_level" input

Invalid inputs to test
1. "approach" = "Time" with a "centered_year" between 2015 and 2099, with "warming_level" input
2. "approach" = "Time" and no "centered_year" provided
3. "centered_year" outside of 2015-2099
4. "approach" that is not "Time" or "Warming Level"

profile_selections = {
    "variable": variable,
    "resolution": "9 km",
    "q": qtile,
    #"warming_level": [1.5],
    "units": units,
    "no_delta": no_delta,
    # approach
    "approach": "Time",
    "centered_year": 2016,
    "time_profile_scenario": "SSP 3-7.0",
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    "latitude": (
        latitude - 0.02,
        latitude + 0.02,
    ),  # uncomment for a using a custom coordinate location
    "longitude": (
        longitude - 0.02,
        longitude + 0.02,
    ),  # uncomment for a custom coordinate location
    # "cached_area": area_name, # uncomment for a cached area
}

results in:

No data found in bounds. Data variable: t2
Skipping spatial subsetting.

In [7]:
profile_selections = {
    "variable": "Precipitation (total)",
    "resolution": "45 km",
    "q": qtile,
    # "warming_level": [1.5],
    "units": units,
    "no_delta": True,
    # approach
    "approach": "Time",
    "centered_year": 1980,
    #"time_profile_scenario": "SSP 5-8.5",
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name,  # uncomment for a cached area
}

In [8]:
profile = get_climate_profile(**profile_selections)

📊 Retrieving climate data...


Data retrieval:   0%|          | 0/1 [00:00<?, ?dataset/s]

You have chosen to produce a time-based Standard Year climate profile centered around 1980. 
Standard year functionality for time-based profiles: 
1. identifies the closest warming level at that centered year for either the input SSP scenario or default 'SSP 3-7.0' 
if no scenario input is provided. And 'historical' if 'centered_year' < 2015. 
2. then filters simulations by either the input SSP scenario or the default 'SSP 3-7-0'. 

Step 1: The corresponding global warming level for input centered year 1980 will now be determined.

Year 1980 before 2015, using Historical data
Corresponding warming level for 'centered_year'= 1980 and SSP 'Historical' is [0.35]. 

Step 2: The climate profile will now be produced using SSP 'SSP 3-7.0' and warming level [0.35]. Only models corresponding to this scenario will be used.
Using default 'q': 0.5
⚙️  Computing climate profiles...
      📊 Processing 87,600 hours (10 years) of data
      🎯 Computing 50th percentile for each hour of year
      ⚙️ Co

      Computing profiles:   0%|          | 0/3 [00:00<?, ?combo/s]

      ✅ Profile computation complete! Final shape: (365, 72)
         With index: Day of Year, columns: ['Hour', 'Simulation']
         Units: degF
   ✓ No baseline subtraction requested, returning raw future profile


In [ ]:
profile

In [ ]:
export_profile_to_csv(profile, **profile_selections)

In [5]:
# retrieve_params = {
#     "variable": variable,
#     "resolution": "3 km",
#     # "q": qtile,
#     #"warming_level": [1.13],
#     "units": units,
#     # "no_delta": False,
#     # # approach
#     "approach": "Time",
#     "centered_year": 2016,
#     "time_profile_scenario": "SSP 3-7.0",
#     # # warming level window
#     "warming_level_window": 5,
#     # # Location options -- uncomment based on your desired location type
#     # "stations": station_name,  # uncomment for a weather station
#     # "latitude": (
#     #     latitude - 0.02,
#     #     latitude + 0.02,
#     # ),  # uncomment for a using a custom coordinate location
#     # "longitude": (
#     #     longitude - 0.02,
#     #     longitude + 0.02,
#     # ),  # uncomment for a custom coordinate location
#     "cached_area": area_name,  # uncomment for a cached area
# }

retrieve_params = {
    'variable': 'Air Temperature at 2m',
    'resolution': '9 km',
    # "warming_level": [1.13],
    'units': 'degF',
    'no_delta': True,
    # approach
    "approach": "Time",
    "centered_year": 2015,
    #'time_profile_scenario': 'SSP 2-4.5',
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name, # uncomment for a cached area
}

In [6]:
historic_data, future_data = retrieve_profile_data(**retrieve_params)

You have chosen to produce a time-based Standard Year climate profile centered around 2015. 
Standard year functionality for time-based profiles: 
1. identifies the closest warming level at that centered year for either the input SSP scenario or default 'SSP 3-7.0' 
if no scenario input is provided. And 'historical' if 'centered_year' < 2015. 
2. then filters simulations by either the input SSP scenario or the default 'SSP 3-7-0'. 

Step 1: The corresponding global warming level for input centered year 2015 will now be determined.

Using default 'SSP 3-7.0' to find corresponding warming level.
Corresponding warming level for 'centered_year'= 2015 and SSP 'SSP 3-7.0' is [1.09]. 

Step 2: The climate profile will now be produced using SSP 'SSP 3-7.0' and warming level [1.09]. Only models corresponding to this scenario will be used.


In [5]:
future_data['simulation']

<xarray.DataArray 'simulation' (simulation: 1)> Size: 168B
array(['WRF_CESM2_r11i1p1f1_historical+ssp245'], dtype='<U42')
Coordinates:
    Lambert_Conformal  int64 8B 0
    centered_year      (simulation) float64 8B 1.988e+03
  * simulation         (simulation) <U42 168B 'WRF_CESM2_r11i1p1f1_historical...
Attributes:
    description:  combined simulation and scenario

In [ ]:
filtered = _filter_by_ssp(future_data,scenario="SSP 2-4.5")

In [ ]:
filtered

In [ ]:
get_gwl_at_year(1980)

In [ ]:
test = {
    "var_id": "t2",
    "q": 0.5,
    "location": "sacramento county",
    "no_delta": False,
    "approach": "Time",
    "centered_year": 2016,
}

In [ ]:
_handle_approach_params(**test)